# Range Bar Generation


## Average ADR over 3 month periods

- csv_path: data/BTCUSDT_1m_3m_back_from_2023-03-14.csv,
- average_adr: 335.17217391304376,
- range_size: 33.51721739130438

---

- csv_path: data/BTCUSDT_1m_3m_back_from_2022-11-15.csv,
- average_adr: 448.0728070175438,
- range_size: 44.80728070175439

---

- csv_path: data/BTCUSDT_1m_3m_back_from_2022-07-19.csv,
- average_adr: 829.0913043478259,
- range_size: 82.9091304347826


Looks like volatility has been decreasing in BTCUSDT

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/BTCUSDT_1m_3m_back_from_2023-03-14.csv')

In [ ]:
df['close'].plot()

In [ ]:
def create_range_bar_df(df: pd.DataFrame, range_size: float):
        range_bars = []
        current_bar = {'timestamp': df.iloc[0]['timestamp'], 'open': df.iloc[0]['open'], 'high': df.iloc[0]['high'], 'low': rb_rb_df.iloc[0]['low'], 'close': df.iloc[0]['close']}
        current_high = current_bar['high']
        current_low = current_bar['low']

        for _, row in df.iterrows():
            high = row['high']
            low = row['low']

            if high - current_low >= range_size:
                current_bar['close'] = current_low + range_size
                range_bars.append(current_bar)

                num_bars = int((high - current_low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'timestamp': row['timestamp'], 'open': current_low + range_size * (i), 'high': current_low + range_size * (i + 1), 'low': current_low + range_size * (i), 'close': current_low + range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'timestamp': row['timestamp'], 'open': current_low + range_size * num_bars, 'high': high, 'low': current_low + range_size * num_bars, 'close': row['close']}
                current_high = high
                current_low = current_bar['low']

            elif current_high - low >= range_size:
                current_bar['close'] = current_high - range_size
                range_bars.append(current_bar)

                num_bars = int((current_high - low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'timestamp': row['timestamp'], 'open': current_high - range_size * (i + 1), 'high': current_high - range_size * (i), 'low': current_high - range_size * (i + 2), 'close': current_high - range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'timestamp': row['timestamp'], 'open': current_high - range_size * (num_bars + 1), 'high': current_high - range_size * num_bars, 'low': low, 'close': row['close']}
                current_high = current_bar['high']
                current_low = low
            else:
                current_high = max(current_high, high)
                current_low = min(current_low, low)
                current_bar['timestamp'] = row['timestamp']
                current_bar['high'] = current_high
                current_bar['low'] = current_low
                current_bar['close'] = row['close']

        return pd.DataFrame(range_bars)

                
            

In [ ]:
rb_df = create_range_bar_df(df, 33.51721739130438)
rb_df.head(20)

check 1: greater then range duplicate starting timestamp rows


In [ ]:
duplicate_rows = rb_df[rb_df.duplicated(subset=['timestamp'])]
duplicate_rows

check 2: less then range size, jumps in 1 min intervals


In [ ]:
# convert timestamp column to timestamp type
rb_df['timestamp'] = pd.to_datetime(rb_df['timestamp'])

# calculate the time difference between each row and the previous row
time_diff = rb_df['timestamp'].diff()

# find consecutive pairs where the time difference is greater than 1 minute
consecutive_pairs = rb_df[(time_diff > pd.Timedelta(minutes=1)) & (time_diff.shift(1) > pd.Timedelta(minutes=1))]
consecutive_pairs

In [ ]:
rb_df['close'].plot()

In [ ]:
rb_df.to_csv('data/range_bars_BTCUSDT_1m_3m_back_from_2023-03-14.csv')